In [2]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching ndw_interference_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s ndw_interference_Scene"

In [ ]:
class SoundSources(Mobject):
    def __init__(self, center_A = np.array([-2.5, -1.5, 0]), center_B = np.array([2.5, -1.5, 0]), c = 300, r0 = 0.75, wave_density = 10, **kwargs):
        super().__init__(**kwargs)

        self.speaker_A = center_A                       # center of speaker A
        self.speaker_B = center_B                       # center of speaker B

        self.c = c                                      # speed of sound
        self.r0 = r0                                    # minimal distance from the sound sources
        self.d = np.sqrt(sum((center_A-center_B)**2))        # distance between the sound sources
        self.wave_density = wave_density                # number wavelines for for dr = 1

        speaker_left = ImageMobject("../external_media/lautsprecher_black.png").scale(0.25).move_to(center_A)
        speaker_right = ImageMobject("../external_media/lautsprecher_black.png").scale(0.25).move_to(center_B)
        self.add(speaker_left, speaker_right)


    # calculates the amplitude of the wave depending on frequency nu, distance from the source r, and time t
    def get_amplitude(self, nu, r, t):
        omega = 2*PI*nu                                 # circle frequency
        k = omega / self.c                              # wave number
        a0 = min(k * self.d / 2, k*self.r0)             # a0 set to limit amplitude to 1
        # print(f"sinarg % pi: {(omega*t- k*r) % PI}")
        return a0 / k / r * (np.sin(omega*t - k*r))
    
    
    # returns the wave field as a group of circles around both sound sources
    def get_sound_waves(self, nu, t):
        # condition for waves: distance bigger than minimum, distance lower than speed of sound times time
        r = self.r0                                     # set r to minimum spread              
        r_max = self.c * t                              # set r_max to maximum spread
        dr = 1.0 / self.wave_density                    # radius increment 
        wave_group = VGroup()                           # group with all wave lines
        # r_max line
        if t > self.r0 / self.c:
            amplitude = self.get_amplitude(nu, r_max, t)
            if amplitude > 0:
                wave_A = Circle(radius = r_max, color = BLACK, stroke_width = 2, stroke_opacity = amplitude).move_to(self.speaker_A)
                wave_B = Circle(radius = r_max, color = BLACK, stroke_width = 2, stroke_opacity = amplitude).move_to(self.speaker_B)
            else:
                wave_A = Circle(radius = r_max, color = WHITE, stroke_width = 2, stroke_opacity = abs(amplitude)).move_to(self.speaker_A)
                wave_B = Circle(radius = r_max, color = WHITE, stroke_width = 2, stroke_opacity = abs(amplitude)).move_to(self.speaker_B)
            wave_group.add(wave_A, wave_B)
        # all other radius lines
        while r < min(r_max, 12):
            amplitude = self.get_amplitude(nu, r, t)    # amplitude of the waves (between 0 and 1)
            if amplitude > 0:
                wave_A = Circle(radius = r, color = BLACK, stroke_width = 2, stroke_opacity = amplitude).move_to(self.speaker_A)
                wave_B = Circle(radius = r, color = BLACK, stroke_width = 2, stroke_opacity = amplitude).move_to(self.speaker_B)
            else:
                wave_A = Circle(radius = r, color = WHITE, stroke_width = 2, stroke_opacity = abs(amplitude)).move_to(self.speaker_A)
                wave_B = Circle(radius = r, color = WHITE, stroke_width = 2, stroke_opacity = abs(amplitude)).move_to(self.speaker_B)
            wave_group.add(wave_A, wave_B)
            # print(r, amplitude)
            r += dr

        return wave_group
    

    # draws a hyperbola from vertex distance a from center between speakers
    def get_hyperbola(self, a, color_hyperbola):
        (x0, y0, z0) = (self.speaker_A + self.speaker_B) / 2             # center between the 2 speakers
        c = self.d / 2                                              # focal point distance to center between speakers
        b = np.sqrt(c**2 - a**2)                                    # co-vertex distance a from center between speakers
        # self.add(Dot(color = BLUE).move_to([a-x0, y0, z0]))         # 200 IQ debugging dots
        # speakers_vector = (self.speaker_B - self.speaker_A) / self.d        # unit vector from speaker A to speaker B
        def hyperbola(x, y):
            smoothing_factor = 10e-5
            return (x-x0)**2 / (a+smoothing_factor)**2 - (y-y0)**2 / (b+smoothing_factor)**2 - 1
        return ImplicitFunction(hyperbola, color = color_hyperbola, stroke_opacity = 0.5)
        # return ImplicitFunction(lambda x, y: (x-x0)**2 / (a-x0) + (y-y0)**2 / (b-y0)**2 - 1)
    

    def get_hyperbola_constructive(self, nu):
        lambda_wavelength = self.c / nu
        constructive_hyperbola_group = VGroup()
        for i in range(int(self.d / lambda_wavelength)):
            a = lambda_wavelength / 2 * i
            constructive_hyperbola = self.get_hyperbola(a, RED)
            constructive_hyperbola_group.add(constructive_hyperbola)
        return constructive_hyperbola_group
    

    def get_hyperbola_destructive(self, nu):
        lambda_wavelength = self.c / nu
        destructive_hyperbola_group = VGroup()
        for i in range(int(self.d / lambda_wavelength) + 1):
            a = lambda_wavelength / 4 * (2*i - 1)
            destructive_hyperbola = self.get_hyperbola(a, BLUE)
            destructive_hyperbola_group.add(destructive_hyperbola)
        return destructive_hyperbola_group

    

# class FrequencyBox(Mobject):
#     def __init__(self):
#         super().__init__(**kwargs)

#             nu_min = 20
#             nu_max = 20000
    

# class Crowd(Mobject):
#     def __init__(self):
#         super().__init__(**kwargs)

In [297]:
%%manim -qh --fps 60 $video_scene


class ndw_interference_Scene(Scene):
    def construct(self):
        self.camera.background_color = GRAY

        # parameters
        nu = 300
        c = 300
        time_max = 10


        sound_source = SoundSources(c = c, wave_density = 40)
        self.add(sound_source)

        sound_waves = sound_source.get_sound_waves(nu, 0)
        self.add(sound_waves)


        # constructive interference hyperbolas
        constructive_interference = sound_source.get_hyperbola_constructive(nu)
        destructive_interference = sound_source.get_hyperbola_destructive(nu)
        # self.add(constructive_interference, destructive_interference)


        def sound_waves_updater(wave):
            t = time_tracker.get_value()
            nu = nu_tracker.get_value()
            wave.become(sound_source.get_sound_waves(nu, t))


        def constructive_interference_updater(interference):
            nu = nu_tracker.get_value()
            interference.become(sound_source.get_hyperbola_constructive(nu))


        def destructive_interference_updater(interference):
            nu = nu_tracker.get_value()
            interference.become(sound_source.get_hyperbola_destructive(nu))


        self.wait(1.5)
        time_tracker = ValueTracker(0)
        nu_tracker = ValueTracker(nu)

        sound_waves.add_updater(sound_waves_updater)

        self.play(time_tracker.animate.set_value(10/c), rate_func = linear, run_time = 10)
        self.play(time_tracker.animate.set_value(13/c), FadeIn(constructive_interference), FadeIn(destructive_interference), rate_func = linear, run_time = 3)

        constructive_interference.add_updater(constructive_interference_updater)
        destructive_interference.add_updater(destructive_interference_updater)

        self.play(time_tracker.animate.set_value(23/c), nu_tracker.animate.set_value(2*nu), rate_func = linear, run_time = 10)

        constructive_interference.remove_updater(constructive_interference_updater)
        destructive_interference.remove_updater(destructive_interference_updater)

        self.play(time_tracker.animate.set_value(26/c), FadeOut(constructive_interference), FadeOut(destructive_interference), rate_func = linear, run_time = 3)


        # self.play(time_tracker.animate.set_value(36/c), rate_func = linear, run_time = 10)
        self.wait(3)

Manim Community v0.18.1